In [1]:
import pandas as pd

# Load your data
df = pd.read_csv("loan_data.csv")

# Quick look at data
print(df.head())
print(df.info())
print(df.describe())

   customer_id  credit_lines_outstanding  loan_amt_outstanding  \
0      8153374                         0           5221.545193   
1      7442532                         5           1958.928726   
2      2256073                         0           3363.009259   
3      4885975                         0           4766.648001   
4      4700614                         1           1345.827718   

   total_debt_outstanding       income  years_employed  fico_score  default  
0             3915.471226  78039.38546               5         605        0  
1             8228.752520  26648.43525               2         572        1  
2             2027.830850  65866.71246               4         602        0  
3             2501.730397  74356.88347               5         612        0  
4             1768.826187  23448.32631               6         631        0  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column                 

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

# Load dataset
df = pd.read_csv("loan_data.csv")

# Features and target
features = ['credit_lines_outstanding', 'loan_amt_outstanding',
            'total_debt_outstanding', 'income', 'years_employed', 'fico_score']

X = df[features]
y = df['default']

# Scale numeric features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Logistic Regression model (baseline)
lr_model = LogisticRegression()
lr_model.fit(X_train, y_train)

# Evaluate model
y_pred_prob = lr_model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_pred_prob))

# Function to compute Expected Loss
recovery_rate = 0.10  # 10% recovery

def expected_loss(loan_features, model=lr_model, scaler=scaler, recovery_rate=recovery_rate):
    """
    loan_features: dict with keys matching 'features'
    Returns expected loss
    """
    # Convert to DataFrame
    df_input = pd.DataFrame([loan_features])
    
    # Scale numeric features
    df_input_scaled = scaler.transform(df_input)
    
    # Predict PD
    pd_pred = model.predict_proba(df_input_scaled)[:, 1][0]
    
    # Exposure = loan amount outstanding
    exposure = df_input['loan_amt_outstanding'].values[0]
    
    # Expected Loss formula
    el = pd_pred * exposure * (1 - recovery_rate)
    return el


ROC-AUC: 0.9999652110990509


In [7]:
loan_example = {
    'credit_lines_outstanding': 5,
    'loan_amt_outstanding': 20000,
    'total_debt_outstanding': 5000,
    'income': 60000,
    'years_employed': 3,
    'fico_score': 710
}

el = expected_loss(loan_example)
print(f"Expected Loss for this loan: {el:.2f}")

Expected Loss for this loan: 17986.71
